In [1]:
from dm_zoo.dff.PixelDiffusion import PixelDiffusionConditional
from WD.io import write_config, load_config
from WD.datasets import Conditional_Dataset_Zarr_Iterable
from WD.utils import check_devices, create_dir, generate_uid, AreaWeightedMSELoss
from pytorch_lightning import loggers as pl_loggers

/home/wider/.conda/envs/TORCH311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds_id = "96FE8A"

print(f"Loading dataset from {ds_id}.yaml")

ds_config_path = f"/data/compoundx/WeatherDiff/config_file/{ds_id}.yml"
ds_config = load_config(ds_config_path)


# datasets:
train_ds_path = ds_config.file_structure.dir_model_input + f"{ds_id}_train.zarr"
train_ds = Conditional_Dataset_Zarr_Iterable(train_ds_path, ds_config_path, shuffle_chunks=True, shuffle_in_chunks=True)

val_ds_path = ds_config.file_structure.dir_model_input + f"{ds_id}_val.zarr"
val_ds = Conditional_Dataset_Zarr_Iterable(val_ds_path, ds_config_path, shuffle_chunks=True, shuffle_in_chunks=True)

Loading dataset from 96FE8A.yaml


In [3]:
lat_grid = train_ds.data.targets.lat[:]
lon_grid =  train_ds.data.targets.lon[:]

loss_fn = AreaWeightedMSELoss(lat_grid, lon_grid).loss_fn 

In [5]:
model = PixelDiffusionConditional(
    train_dataset=train_ds,
    valid_dataset=val_ds,
    generated_channels=ds_config.n_generated_channels,
    condition_channels=ds_config.n_condition_channels,
    batch_size=64,
    cylindrical_padding=True,
    lr=1e-4,
    num_workers=4,
    loss_fn=loss_fn,
    lr_scheduler_name="Constant"
)

KeyError: 'ipython_dir'